In [ ]:
import datacube
from datacube.storage import masking
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

In [ ]:
dc = datacube.Datacube(app='agriculture-app')
dc.list_products()

In [ ]:
latitude = (-34.426512, -34.434517)
longitude = (116.648123, 116.630731)
time = ("2015-01-01", "2018-12-31")

In [ ]:
dataset_fc = dc.load(product='ls8_fc_albers', latitude=latitude, longitude=longitude, time=time)
dataset_wofs = dc.load(product='wofs_albers', latitude=latitude, longitude=longitude, time=time, output_crs='epsg:3577', resolution = (-25,25))

In [ ]:
shared_times = np.intersect1d(dataset_fc.time, dataset_wofs.time)

ds_fc_matched = dataset_fc.sel(time=shared_times)
ds_wofs_matched = dataset_wofs.sel(time=shared_times)

In [ ]:
dry_mask = masking.make_mask(ds_wofs_matched, dry=True)

In [ ]:
ds_fc_masked = ds_fc_matched.where(dry_mask.water == True)

In [ ]:
scene=3

plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
ds_fc_masked.PV.isel(time=scene).plot(cmap='gist_earth_r')
ax1.set_title('PV')

ax2=plt.subplot(gs[1,0])
ds_fc_masked.BS.isel(time=scene).plot(cmap='Oranges')
ax2.set_title('BS')

ax3=plt.subplot(gs[0,1])
ds_fc_masked.NPV.isel(time=scene).plot(cmap='copper')
ax3.set_title('NPV')

ax4=plt.subplot(gs[1,1])
ds_fc_masked.UE.isel(time=scene).plot(cmap='magma')
ax4.set_title('UE')

plt.tight_layout()
plt.show()